[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/talmolab/dreem/blob/docs/examples/quickstart.ipynb)

# DREEM Quickstart: Fly Tracking Demo

Welcome to DREEM! This notebook will guide you through tracking a social interaction between two flies using a pretrained model. By the end, you'll have hands-on experience with the core DREEM workflow.

### What you'll learn:
- How to download sample data and pretrained models
- Running tracking inference with `dreem track`
- Evaluating tracking accuracy with `dreem eval`
- Visualizing tracking results

### Requirements:
- **Runtime**: ~5-10 minutes total
- **Hardware**: CPU is sufficient
- **Data**: We provide sample fly videos from the [SLEAP fly32 dataset](https://sleap.ai/datasets.html#fly32)


---
## Step 1: Install Dependencies

First, we'll install DREEM and the required packages. This may take a few minutes.


In [ ]:
# Install uv (fast Python package installer)
%pip install uv

# Install DREEM and dependencies using uv
!uv pip install dreem-track huggingface_hub opencv-python-headless

# Install ffmpeg for video visualization (Colab-specific)
!apt-get install -y ffmpeg 2>/dev/null || echo "ffmpeg already installed or not on Linux"

### Check available hardware

Let's detect what compute device is available. DREEM works on CPU, CUDA (NVIDIA GPUs), or MPS (Apple Silicon).


In [1]:
import torch

cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    accelerator = "cuda"
    print(f"GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    accelerator = "mps"
    print("Using Apple Silicon (MPS)")
else:
    accelerator = "cpu"
    print("Using CPU (tracking will be slower but still works!)")

print(f"\n✓ Device selected: {accelerator}")
torch.set_float32_matmul_precision("medium")


CUDA available: False
Using Apple Silicon (MPS)

✓ Device selected: mps


---
## Step 2: Download Sample Data

We'll download a sample fly tracking dataset from Hugging Face. This includes:
- Video files (`.mp4`) of fly interactions
- Detection files (`.slp`) with pose keypoints from SLEAP
- Configuration files for running inference


In [23]:
!hf download talmolab/sample-flies --repo-type dataset --local-dir ./data


A new version of huggingface_hub (1.3.1) is available! You are using version 1.2.4.
To update, run: brew upgrade huggingface-cli

Fetching 12 files:   0%|                                 | 0/12 [00:00<?, ?it/s]Still waiting to acquire lock on data/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on data/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Fetching 12 files: 100%|████████████████████████| 12/12 [00:15<00:00,  1.30s/it]
Download complete: : 260MB [00:15, 15.1MB/s]              /Users/mustafashaikh/dreem-docs/examples/data
Download complete: : 260MB [00:15, 16.7MB/s]


---
## Step 3: Download Pretrained Model

Now we'll download a pretrained DREEM model. This model was trained on various animal data (mice, flies, zebrafish) and generalizes well to new animal tracking tasks.


In [5]:
!hf download talmolab/animals-pretrained animals-pretrained.ckpt --local-dir=./models


animals-pretrained.ckpt: 100%|███████████████| 139M/139M [00:04<00:00, 33.2MB/s]
models/animals-pretrained.ckpt


---
## Step 4: Run Tracking

We have filled in appropriate values for crop size and max tracks (2-fly dataset). In general, you will have to select an appropriate crop size for your data. The crop size is the size of the bounding box (in pixels) around the instance that is used for tracking.

**What happens during tracking:**
1. DREEM loads the video and detections (pose keypoints)
2. For each frame, it extracts visual features around each detection
3. The transformer model associates detections across frames to form tracks
4. Results are saved as a new `.slp` file with track IDs assigned


In [ ]:
!dreem track ./data/inference --checkpoint ./models/animals-pretrained.ckpt --output ./results --crop-size 70 --max-tracks 2 


[2026-01-07 22:06:35,651][dreem.io][INFO] - Base Config: {'ckpt_path': './models/animals-pretrained.ckpt', 'tracker': {'overlap_thresh': 0.01, 'decay_time': None, 'iou': 'mult', 'max_tracks': 2, 'max_center_dist': None, 'persistent_tracking': True}, 'dataset': {'test_dataset': {'dir': {'path': './data/inference', 'labels_suffix': '.slp', 'vid_suffix': '.mp4'}, 'chunk': True, 'clip_length': 32, 'anchors': 'centroid'}}, 'dataloader': {'test_dataloader': {'shuffle': False, 'num_workers': 0}}, 'trainer': {'limit_test_batches': 1.0}}
Using the following tracker:
[2026-01-07 22:06:35,896][dreem.models][INFO] - Using the following tracker:
Tracker(max_tracks=2, use_vis_feats=True, overlap_thresh=0.01, mult_thresh=True, decay_time=None, max_center_dist=inf, verbose=False, queue=TrackQueue(window_size=8, max_gap=inf, n_tracks=0, queues=[], curr_gap:{}), temperature=0.1queue=TrackQueue(window_size=8, max_gap=inf, n_tracks=0, queues=[], curr_gap:{}), temperature=0.1
[2026-01-07 22:06:35,896][dree

---
## Step 5: Evaluate Tracking Accuracy (Optional)

If you have ground truth labels, you can use `dreem eval` instead of `dreem track`. This computes tracking metrics like:
- **MOTA** (Multiple Object Tracking Accuracy)
- **IDF1** (ID F1 Score)  
- **Number of ID switches**


In [ ]:
!dreem eval ./data/test --checkpoint ./models/animals-pretrained.ckpt --output "./eval-results" --crop-size 70 --max-tracks 2 

Configuration saved to: eval-results-3/config.eval.02-04-2026-11-45-11.yaml
╭───────────────────────────── Eval Configuration ─────────────────────────────╮
│ Checkpoint:  /Users/main/Downloads/epoch=15-val_loss=0.5974664688110352.ckpt │
│ Output:      eval-results-3                                                  │
│ Input:       data/test                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯
Testing ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80/80 0:03:14 • 0:00:00 0.42it/s it/s it/s 
Evaluation complete.


---
## Step 6: Visualize Results

Let's visualize the tracking results! We'll create an animation showing the tracked flies with their assigned IDs.

> **Note**: For the best visualization experience with full pose keypoints, you can open the `.slp` file in the SLEAP GUI locally:
> ```bash
> sleap-label results/<your_output_file>.slp
> ```
> The SLEAP GUI won't render in Colab, but the animation below gives you a quick preview.


In [ ]:
import os
import sleap_io as sio
import pandas as pd
import numpy as np
from create_animation import create_tracking_animation

### Load tracking results


In [8]:
# Find the latest tracking result
results_dir = "./results"
result_files = sorted([f for f in os.listdir(results_dir) if f.endswith('.slp') and 'dreem_inference' in f])
result_path = os.path.join(results_dir, result_files[-1])
print(f"Loading results from: {result_path}")

# Load the predictions
pred_slp = sio.load_slp(result_path)
print(f"Loaded {len(pred_slp)} frames with tracking results")


Loading results from: ./results/190719_090330_wt_18159206_rig1.2@15000-17560.dreem_inference.02-04-2026-10-49-47.slp
Loaded 2560 frames with tracking results


In [9]:
# Convert predictions to a DataFrame for visualization
list_frames = []
for lf in pred_slp:
    for instance in lf.instances:
        centroid = np.nanmean(instance.numpy(), axis=0)
        track_id = int(instance.track.name) if instance.track else -1
        list_frames.append({
            "frame_id": lf.frame_idx,
            "track_id": track_id,
            "centroid": centroid
        })
df = pd.DataFrame(list_frames)

### Create tracking animation


In [ ]:
# Find the video file used for inference
video_dir = "./data/inference"
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
video_path = os.path.join(video_dir, video_files[0])
print(f"Video: {video_path}")

# Create the animation
anim = create_tracking_animation(video_path=video_path, metadata_df=df, fps=15, marker_size=15, max_frames=100)

Video: ./data/inference/190719_090330_wt_18159206_rig1.2@15000-17560.mp4
Showing first 100 frames
Creating animation with 100 frames...


---
## Next Steps

Congratulations! You've successfully run DREEM tracking on fly data. Here's where to go next:

### Dive Deeper
- **[End-to-End Demo](https://colab.research.google.com/github/talmolab/dreem/blob/docs/examples/dreem-demo.ipynb)**: Train your own model, run inference, and evaluate results
- **[Microscopy Demo](https://colab.research.google.com/github/talmolab/dreem/blob/docs/examples/microscopy-demo-simple.ipynb)**: Track cells in microscopy data

### Documentation
- **[Usage Guide](https://dreem.sleap.ai/usage/)**: Full CLI reference and configuration options
- **[Configuration Reference](https://dreem.sleap.ai/configs/)**: Customize training and inference parameters
- **[API Reference](https://dreem.sleap.ai/reference/dreem/)**: Python API documentation

### Use Your Own Data
The pretrained animals model works with any SLEAP detections! Just:
1. Generate pose predictions with [SLEAP](https://sleap.ai/)
2. Update the config to point to your video and `.slp` files
3. Run `dreem track` as shown above

Happy tracking! 🪰
